**Table of contents**<a id='toc0_'></a>    
- [Feature selection](#toc1_1_)    
    - [Principle Component Analysis](#toc1_1_1_)    
    - [Feature importance from tree-based models (e.g., Random Forest)](#toc1_1_2_)    
- [model](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, HTML
import sys
import os

import warnings
warnings.filterwarnings("ignore")
sys.path.append('/Users/jianbinlin/Documents/Master-Data-Spreadsheet/script/')

import utility
from utility import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

display(HTML("<style>.container { width:50% !important; }</style>"))
#display(HTML('<style>.output { max-width:800px !important; }</style>')) # control output width
display(HTML("<style>.container { width:130% !important; }</style>"))

pd.set_option('display.max_columns', None)   # Show number of columns, None means show all columns
pd.set_option('display.max_rows', 50)        # show top and bottom 15, middle with ......
pd.set_option('display.width', None)         # Set the width of the display
pd.set_option('display.max_colwidth', None)  # Show full content of each column

24/11/30 16:47:18 WARN Utils: Your hostname, Jians-iMac.local resolves to a loopback address: 127.0.0.1; using 10.0.0.23 instead (on interface en1)
24/11/30 16:47:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/30 16:47:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
credit_card = pd.read_csv('~/Desktop/creditcard.csv')
credit_card.tail(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [35]:
def csvfile_chunk(df,n,file_location):
    """  
    1. take a csv file and divide into n chunks
    """
    for i in range(n):
        number_sections = int(len(df) / n)
        section1 = number_sections*i 
        section2 = number_sections*(i+1)
        section = df.iloc[(int(len(df) / n))*i:(int(len(df) / n))*(i+1),:]
        section.to_csv(file_location+'df_'+str(i)+'.csv')
        if i+1 == n:
            section = df.iloc[(int(len(df) / n))*(i+1):(int(len(df) / n))*(i+2),:]
            section.to_csv(file_location+'df_'+str(i+1)+'.csv')

        

        # print(number_sections,i,section1,section2)
        #df_str(i) =df.iloc[]

csvfile_chunk(credit_card,5,'/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/')        

In [34]:
for i in range(5):
    print(i)

0
1
2
3
4


In [29]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_0.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56956,47693.0,1.244207,0.485800,-0.537025,1.049686,0.280647,-0.516701,0.072789,0.036030,-0.056136,-0.349735,0.856880,-0.428801,-1.566466,-0.584493,0.581167,0.974828,0.262168,1.378539,-0.023567,-0.172898,-0.063570,-0.221585,-0.197832,-0.508676,0.699926,-0.297100,0.016902,0.030725,1.00,0
56957,47693.0,-0.662015,0.401111,1.429916,1.364612,0.104361,-0.169103,0.861198,-0.005588,-0.459026,-0.438143,-0.311174,0.536188,0.282514,-0.020275,-0.056034,-1.198422,0.592997,-0.824562,0.287084,0.257749,0.136696,0.399885,0.206171,0.423881,-0.255291,-0.343978,0.157792,0.176427,111.10,0
56958,47693.0,-2.059453,1.784619,1.424784,-0.593855,-0.226328,-1.430378,0.882338,0.195611,-0.293406,-0.473571,-0.252622,0.358266,-0.140165,0.288841,-0.261944,0.555565,-0.708557,-0.549579,-2.041479,-0.200014,-0.072851,-0.408204,-0.160125,0.871726,0.691075,-0.814963,-0.173250,0.137558,15.20,0
56959,47694.0,-0.695471,0.933852,1.122342,-1.160664,-0.636954,-1.163185,0.124088,0.419856,0.031530,-0.862240,-0.716799,-0.179890,-0.561789,0.356915,0.545417,0.802161,-0.490589,-0.258013,-0.758827,-0.150196,-0.074592,-0.321212,0.028948,0.403167,-0.252059,0.727798,0.047921,0.003653,0.77,0


In [30]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_1.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
56961,47694.0,1.138149,-0.698637,0.332976,0.272394,-0.629432,0.463714,-0.628447,0.096643,-1.013536,0.952361,0.034845,0.333348,0.835304,0.054927,0.433435,-0.510982,-1.045337,2.400609,-0.693835,-0.242242,-0.200924,-0.328477,-0.240835,-0.868956,0.483849,-0.256578,0.045728,0.034950,116.00,0
56962,47695.0,0.017199,-0.148533,-0.095542,-0.923477,1.161514,-0.560818,0.874059,-0.141331,-0.241034,-0.073535,1.635645,0.927543,-0.223847,0.458696,-0.390453,-0.283662,-0.385551,-1.288478,-0.936738,-0.021917,0.034412,0.099442,0.785986,-0.291115,-2.653623,0.148533,0.101011,0.124571,55.98,0
56963,47695.0,1.157218,-0.400497,0.224997,-0.662346,-0.058859,0.701721,-0.468211,0.178250,0.147136,-0.162543,0.721268,1.240581,1.678431,-0.116679,0.909300,0.881945,-0.956743,-0.031011,0.413422,0.204080,-0.082828,-0.319259,-0.084150,-1.238330,0.039034,1.321246,-0.076822,-0.002819,69.99,0
56964,47695.0,1.227454,0.052971,0.074560,1.151042,-0.047766,0.092690,-0.109024,0.155374,0.385462,0.147932,-0.128367,-0.169250,-1.989227,0.569533,-0.774572,0.096744,-0.454952,0.472139,0.605543,-0.251456,-0.156081,-0.366528,-0.180845,-0.542113,0.751469,-0.301263,0.006493,-0.003925,5.37,0
56965,47695.0,1.228671,1.260991,-1.699439,1.450723,1.079511,-1.310171,0.683609,-0.232911,-0.722463,-1.388395,2.265970,0.234249,0.091488,-3.252825,0.366724,1.317110,2.321057,1.905703,-0.424509,0.001490,-0.177758,-0.433050,-0.265018,-0.320238,0.880613,-0.291865,0.037405,0.083940,0.89,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113917,73260.0,-0.480434,0.884576,1.384364,0.035791,0.004768,-0.883443,0.447147,0.135624,-0.488307,-0.297319,1.476861,0.090118,-1.120622,0.007872,0.305948,0.568827,-0.101735,0.430625,0.130739,0.073864,-0.202767,-0.638256,0.030125,0.453385,-0.251325,0.045658,0.231083,0.090056,1.29,0
113918,73260.0,1.233144,0.189868,0.629888,0.707835,-0.719922,-1.200394,0.011885,-0.177277,0.155055,-0.041299,-0.072763,0.011393,-0.617850,0.433658,1.104276,0.248768,-0.254034,-0.541172,-0.236810,-0.156326,-0.244927,-0.756894,0.185822,0.682525,0.143524,0.077069,-0.034694,0.020766,3.59,0
113919,73260.0,-1.331350,0.872466,0.378659,-1.741660,-0.975754,-0.934276,-0.402536,0.696671,-1.337704,0.021926,-1.500511,-0.364743,0.198200,0.489586,0.267426,-0.646194,-0.203810,1.193958,-1.181160,-0.660344,-0.346029,-0.808934,0.135682,-0.131309,-0.503984,0.712595,-0.198555,0.054277,12.54,0
113920,73260.0,1.124264,0.173281,1.085010,1.055675,-0.719828,-0.404515,-0.315841,0.043801,-0.037384,0.099770,1.625214,1.386238,0.731936,0.177197,0.499164,0.594852,-0.831747,0.388040,-0.290139,-0.057092,0.020557,0.055327,0.054326,0.523417,0.283857,-0.545049,0.048752,0.032138,13.99,0


In [31]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_2.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
113922,73261.0,-0.801496,1.304044,1.457980,0.033394,-0.414816,-1.080460,0.593126,0.064049,-0.414219,-0.226315,-0.108828,0.271175,0.286202,0.178058,0.793307,0.119046,-0.233035,-0.489041,-0.109156,0.048985,-0.196181,-0.496252,0.047429,0.698505,-0.240106,0.011543,-0.154010,-0.165248,9.99,0
113923,73262.0,1.288481,-1.448779,0.823177,-0.905435,-1.856666,-0.308069,-1.088356,-0.149362,-1.227770,1.098964,-0.896082,0.286386,1.752008,-1.302109,-1.483715,-1.190336,1.067697,-0.488155,0.247158,-0.084421,-0.218475,-0.054046,-0.131283,0.459550,0.544345,-0.105002,0.057234,0.039619,104.24,0
113924,73262.0,-1.205377,1.655467,0.558397,0.791684,-0.389180,-0.231137,-0.121880,0.978861,-1.059248,-0.224401,1.274492,0.957490,-0.155478,1.175022,0.501783,-0.251687,0.251166,0.168427,0.250152,-0.149408,0.298652,0.719230,-0.006282,0.232402,-0.293870,-0.324922,0.047809,0.077171,1.23,0
113925,73262.0,-5.207031,4.713869,-3.193733,-0.185909,-2.733456,-1.096451,-2.507009,3.849363,-0.455522,-0.092157,-1.955823,1.632157,1.700835,1.632494,0.637549,1.530722,1.098670,0.237935,-0.071631,0.231026,-0.172883,-1.065294,0.363694,-0.527592,0.589110,0.175566,-0.024710,-0.009260,13.56,0
113926,73263.0,1.251282,0.359359,-0.006559,0.466214,0.279435,-0.153794,0.092639,-0.095660,-0.122345,-0.367878,-0.268386,0.783497,1.655262,-0.563916,1.254914,0.336639,-0.065895,-0.672001,-0.294578,0.010292,-0.288127,-0.744756,0.046990,-0.656000,0.282130,0.163905,-0.000267,0.024146,8.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170878,120394.0,2.085243,0.153914,-1.937797,0.455977,0.457744,-1.599438,0.828357,-0.606184,0.266940,-0.025940,-0.720457,0.728269,0.480436,0.523950,-0.283934,-0.927651,-0.136301,-0.659057,0.148734,-0.211955,0.146098,0.674611,-0.092485,0.116184,0.578814,-0.074287,-0.049963,-0.071206,13.12,0
170879,120395.0,1.941601,-0.105717,-2.025099,0.021689,1.257679,1.180161,-0.212451,0.406611,0.369452,-0.321224,1.160582,0.526688,-0.682991,-0.456742,0.385901,-0.168836,0.893466,-0.893428,-0.517444,-0.266748,-0.275532,-0.651069,0.353144,-1.058401,-0.454839,0.282256,-0.022103,-0.058297,1.98,0
170880,120395.0,-0.184060,1.192288,-0.831053,-0.379134,0.898165,-0.494622,0.780753,-0.873814,-0.297296,-0.497556,0.801451,-0.261693,-1.664352,-0.373477,-0.747409,0.165447,0.595165,0.619075,0.543861,-0.351765,0.755613,-0.315061,0.223707,0.640872,-1.259095,0.179031,0.105884,0.271658,17.60,0
170881,120395.0,1.418125,-1.676690,-0.942455,0.734790,-0.915707,0.286333,-0.420231,0.070484,-0.475192,1.032365,0.442776,0.376567,-0.370535,0.318471,-0.373292,-1.130296,-0.488532,1.883896,-1.137394,-0.025106,-0.078157,-0.408628,0.023694,0.628874,-0.337955,-0.753715,-0.002207,0.020888,327.60,0


In [32]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_3.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
170883,120396.0,-2.410892,-1.264333,-0.787460,-0.212152,1.139859,-2.030200,0.533818,-0.194852,-2.277774,0.310404,-1.211770,-0.805137,-0.503570,1.154627,0.260146,-2.558533,0.876103,0.902372,0.422115,-0.558831,-0.076699,0.054816,-1.243901,0.147148,0.974330,1.309377,0.083660,-0.611166,124.98,0
170884,120396.0,-1.394526,0.284790,2.979226,0.362345,-1.344546,2.278858,-1.836101,-1.797895,0.529802,0.776728,-0.864413,0.060070,-1.102005,-1.195983,-1.860611,-1.959586,0.448597,2.480298,0.527285,-0.807798,1.945416,-0.539926,-0.446763,-0.521142,0.405680,-0.149237,0.458944,-0.071516,50.00,0
170885,120396.0,0.140378,0.989574,-0.506447,0.480553,1.407763,0.371637,0.811775,0.067702,-0.648266,0.376802,-0.046308,0.167851,0.095213,0.621540,0.065680,-0.081217,-0.895254,1.031398,0.843741,0.120841,0.385872,1.302941,-0.177157,-0.192410,-0.724567,-0.433276,0.540947,0.323866,8.69,0
170886,120396.0,-1.303150,0.187865,0.944395,-1.277834,1.236744,-1.301696,1.402234,-0.558914,-0.041604,-0.694289,-0.806051,0.277925,0.343193,-0.268240,-0.927519,0.153715,-0.885167,-1.040416,-1.133255,-0.251346,-0.274175,-0.656826,-0.164985,0.001284,-0.001842,-0.024050,-0.436872,0.051705,59.99,0
170887,120396.0,1.947883,-1.095332,-1.392121,-0.600598,-0.320840,0.048953,-0.443486,-0.081011,-0.634128,0.975375,0.226406,0.744480,0.852910,0.083724,-0.606380,-1.144119,-0.580166,1.559673,-0.343759,-0.293448,-0.250594,-0.385151,0.099030,0.221922,-0.212846,0.586637,-0.071616,-0.045475,119.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227839,145244.0,0.099222,-0.478594,-0.550868,-2.941157,1.978146,3.547492,-0.826766,0.992349,-0.828225,0.265354,0.040421,-0.788064,0.081494,-0.269084,0.784979,1.467066,-0.302142,-1.355776,0.093827,0.032180,0.090338,0.019818,0.330187,0.658491,-0.686655,-0.537361,-0.044617,-0.012715,4.15,0
227840,145244.0,2.028950,-0.374089,-1.268051,0.349127,-0.057977,-0.515489,-0.087045,-0.146316,1.001341,0.007773,-1.307879,-0.003230,-0.684642,0.109699,-0.461828,-0.330118,-0.087021,-0.437197,0.222290,-0.194318,0.047287,0.323331,0.080683,0.727965,0.070118,0.723291,-0.083032,-0.061277,24.95,0
227841,145245.0,0.306600,1.116021,-0.047348,3.593785,2.079047,5.748707,-2.059246,-1.352120,-1.849240,1.000154,-0.330464,-0.202383,-0.129313,0.713813,1.309414,0.399616,0.038217,-0.097967,0.632624,0.865457,-1.458527,-0.234478,0.258831,0.639005,-0.048250,0.144586,0.216385,0.304545,0.76,0
227842,145245.0,-1.781954,2.062680,-3.758871,-1.801001,-0.084365,1.919610,-1.454364,-9.825473,-2.073119,-0.005816,-0.644403,0.982035,0.243801,1.541216,-1.949005,-1.278610,-0.179722,1.616439,-0.740407,-0.690810,1.234747,0.376804,-0.339706,-0.216688,-1.555393,0.655873,0.777662,0.272753,359.28,0


In [33]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_4.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
227844,145247.0,2.050034,-0.103557,-1.204713,0.207198,0.108850,-0.665621,0.073598,-0.164464,0.303531,0.242471,0.614559,1.016880,0.081998,0.441357,-0.635926,0.203137,-0.682642,-0.251580,0.582943,-0.201782,-0.263799,-0.654763,0.280573,-0.405691,-0.274672,0.202719,-0.074529,-0.074279,0.17,0
227845,145248.0,1.914027,-0.490068,-0.326111,0.604711,-0.850136,-0.736319,-0.524058,-0.088614,1.091125,0.093484,-0.892305,0.027221,-0.243790,0.031774,0.900624,0.536032,-0.648408,0.183072,-0.486322,-0.139579,0.210958,0.639338,0.147523,0.073654,-0.318378,0.350612,-0.023843,-0.037139,50.00,0
227846,145249.0,2.152696,-0.036161,-2.231811,0.091766,0.537612,-1.368103,0.613327,-0.455252,0.291814,0.253161,-1.501882,-0.870608,-1.441738,0.988757,0.496349,-0.068699,-0.454073,-0.299095,0.267443,-0.275778,0.017153,0.063242,-0.034561,-0.626866,0.249213,0.773931,-0.137115,-0.090611,14.95,0
227847,145249.0,-4.034795,2.305079,-1.461693,-0.729887,-1.528750,-1.225679,-0.893354,1.622522,1.291998,-0.040956,-0.971425,0.574744,0.155656,-0.729055,0.477439,1.061719,0.934695,0.403769,-0.494930,-0.081093,-0.392557,-0.787599,0.343468,-0.090331,0.248287,-0.238524,0.266484,-0.062236,7.70,0
227848,145249.0,-1.668741,1.168055,0.249642,-1.268497,0.785923,-0.663959,0.859433,0.068111,-0.144183,0.043288,0.542014,1.002025,0.400760,0.136412,-1.289649,0.276828,-0.868492,-0.366840,-0.187392,-0.033523,-0.247544,-0.592537,-0.286694,-0.378856,-0.077429,0.067608,-0.278962,-0.064193,6.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284800,172784.0,2.039560,-0.175233,-1.196825,0.234580,-0.008713,-0.726571,0.017050,-0.118228,0.435402,0.267772,0.523316,0.559047,-0.834660,0.626211,-0.541494,0.225361,-0.605252,-0.163049,0.561695,-0.256922,-0.268048,-0.717211,0.297930,-0.359769,-0.315610,0.201114,-0.080826,-0.075071,2.68,0
284801,172785.0,0.120316,0.931005,-0.546012,-0.745097,1.130314,-0.235973,0.812722,0.115093,-0.204064,-0.657422,0.644837,0.190916,-0.546329,-0.731707,-0.808036,0.599628,0.070441,0.373110,0.128904,0.000676,-0.314205,-0.808520,0.050343,0.102800,-0.435870,0.124079,0.217940,0.068803,2.69,0
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0


In [36]:
a = pd.read_csv('/Users/jianbinlin/Documents/ML_Classification_Models/Binary/credit_Card_Fraud_Detection/Data/df_5.csv',index_col=0)
a

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.0,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.0,0


In [44]:
credit_card_default = pd.read_csv('https://raw.githubusercontent.com/KevinJianLin/Credit_Card_Default/refs/heads/main/Data/UCI_Credit_Card.csv')
data_target = credit_card_default.copy()
df_profiling_credit_card_default = data_profiling(data_target)
df_profiling_credit_card_default.rest_columns
# df_profiling()

[]

In [45]:
data_target.head(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [46]:
data_target['default.payment.next.month']

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default.payment.next.month, Length: 30000, dtype: int64

In [41]:
label_encoder = LabelEncoder()
data_target['default.payment.next.month'] = label_encoder.fit_transform(data_target['default.payment.next.month'])


In [42]:
# Prepare the feature matrix and target
X = data_target[df_profiling_credit_card_default.float_column + df_profiling_credit_card_default.int_column +df_profiling_credit_card_default.cat_col].drop(columns =['default.payment.next.month'])

y = data_target['default.payment.next.month']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [43]:
y

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default.payment.next.month, Length: 30000, dtype: int64

In [20]:
X_train.head(5)

,LIMIT_BAL,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,ID,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
21753,80000.0,75125.0,77353.0,78321.0,73731.0,39643.0,39457.0,3503.0,5001.0,2092.0,1218.0,1445.0,878.0,21754,2,2,2,24,0,0,0,0,0,0,2,2,2,0,0,0,0,0,0
251,30000.0,29242.0,29507.0,29155.0,25255.0,22001.0,0.0,5006.0,1244.0,851.0,955.0,0.0,0.0,252,1,2,2,28,0,0,0,0,0,0,1,2,2,0,0,0,0,0,0
22941,180000.0,20916.0,0.0,850.0,0.0,6881.0,10340.0,0.0,850.0,0.0,6881.0,10340.0,182.0,22942,2,5,1,44,0,0,-1,-1,-1,-1,2,5,1,0,0,-1,-1,-1,-1
618,60000.0,58839.0,53235.0,38533.0,39639.0,39619.0,39140.0,2018.0,1900.0,2000.0,1500.0,1900.0,2000.0,619,1,1,2,25,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0
17090,130000.0,111587.0,112348.0,114734.0,117823.0,120854.0,123904.0,4100.0,4200.0,5000.0,5000.0,5000.0,10700.0,17091,2,2,2,25,0,0,0,0,0,0,2,2,2,0,0,0,0,0,0


In [21]:
y_train.head(5)

21753    0
251      0
22941    0
618      0
17090    0
Name: default.payment.next.month, dtype: int64

## <a id='toc1_1_'></a>[Feature selection](#toc0_)

### <a id='toc1_1_1_'></a>[Principle Component Analysis](#toc0_)

### <a id='toc1_1_2_'></a>[Feature importance from tree-based models (e.g., Random Forest)](#toc0_)
Permutation importance
SHAP (SHapley Additive exPlanations) values
Recursive feature elimination
Lasso regularization

In [22]:
df_profiling_credit_card_default.float_column

['LIMIT_BAL',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [23]:
df_profiling_credit_card_default.int_column

['ID',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6']

### 

# <a id='toc2_'></a>[model](#toc0_)

In [24]:
text_col =[]
float_col = df_profiling_credit_card_default.float_column 
#int_col = df_profiling_credit_card_default.int_column.remove('default.payment.next.month') 
cat_col =df_profiling_credit_card_default.cat_col

data_preprocessor_1 = ColumnTransformer(transformers = [
                                   #         ('text', TfidfVectorizer(max_features=500), text_col),
                                            ('float_col',StandardScaler(),float_col),
                                        #    ('cat_col', OneHotEncoder(), cat_col),
                                            ]) 

In [30]:
label_encoder.classes_

array([0, 1])

In [25]:
best_model ={}

In [38]:
# Loop through models and apply GridSearchCV
for model_name, model_infor in model_parameters_classification.items():


    # Create a pipeline for each model
    pipeline = Pipeline([('preprocessor',data_preprocessor_1),
                         ('classifier',model_infor['model'])])
    
    # Initialize GridSearchCV with model and parameter grid
    grid_search = GridSearchCV(estimator=pipeline,
                                param_grid = model_infor['params'],
                                cv = 5, # 5 fold cv
                                scoring = 'accuracy', # f1 etc
                                verbose =0, # not to display output
                                n_jobs = -1 # use all of cores
                                )
    grid_search.fit(X_train, y_train)

    best_model[model_name] = grid_search.best_estimator_
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")

    y_pred = grid_search.predict(X_test)
    print(f"Classification Report for {model_name}:\n")
    print(classification_report(y_test, y_pred, target_names = label_encoder.classes_))

    # print(model_infor['model'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('float_col',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PAY_AMT4',
                                                                          'PAY_AMT5',
                                                                          'PAY_AMT6'])])),
                                       ('classifier',
                                        DecisionTreeClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10]},
             scoring='accuracy')

Best Parameters for decisiontree_classifier: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5}
Classification Report for decisiontree_classifier:



TypeError: object of type 'numpy.int64' has no len()

In [36]:
y_test

2308     0
22404    0
23397    0
25058    0
2664     1
        ..
2210     0
14144    1
23108    0
25703    0
29171    0
Name: default.payment.next.month, Length: 6000, dtype: int64

In [37]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])